<h1 align="center"><font color="red">LangGraph: Construindo Aplicações Cíclicas com LangChain</font></h1>

<font color="yellow">Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Este Notebook foi baseado no tutorial de [AI Makerspace]():

Speakers: 

[Dr. Greg, Co-Founder & CEO]()

[The Wiz, Co-Founder & CTO]()

</font color="orange">LangGraph é uma ferramenta que aproveita a linguagem de expressão LangChain para construir aplicativos coordenados com vários atores e com estado que incluem comportamento cíclico.</font>

## Por que Ciclos?

Em essência, podemos pensar em um ciclo em nosso `Graph` como um loop mais robusto e personalizável. Isso nos permite manter nosso aplicativo avançado enquanto ainda oferece a poderosa funcionalidade dos loops tradicionais.

Devido à inclusão de ciclos sobre loops, também podemos compor fluxos bastante complexos através do nosso `Graph` de uma forma muito mais legível e natural. Permitindo-nos efetivamente recriar fluxogramas de aplicativos em código de uma forma quase `1-to-1`.

## Por que LangGraph?

Além da abordagem de agente direto - podemos facilmente compor e combinar cadeias tradicionais `"DAG"` (Directed Acyclic Graph) com comportamento cíclico poderoso devido à forte integração com `LCEL` ([LangChain Expression Language](https://python.langchain.com/v0.1/docs/expression_language/)). Isso significa que é uma extensão natural das principais ofertas da `LangChain`!

## Dependências

Pegaremos algumas dependências e adicionaremos algumas variáveis ​​de ambiente que aproveitaremos em todo o notebook.

In [1]:
%pip install -qU langchain langchain_openai langgraph arxiv duckduckgo-search

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
seaplane-cli 0.0.22b1 requires click==8.1.3, but you have click 8.1.7 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import os
# import getpass

# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key  = os.environ['OPENAI_API_KEY']
#Eddy_key_openai  = os.environ['OPENAI_API_KEY']
#from openai import OpenAI
#client = OpenAI(api_key=Eddy_key_openai)

# Usando LangSmith:
LANGCHAIN_TRACING_V2="true"
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"  # Eddy ---> https://smith.langchain.com/o/fc23d72c-9360-5a5f-affa-26c44b810011
LANGCHAIN_API_KEY="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"
LANGCHAIN_PROJECT="LangGraph_EddyGiusepe"

In [ ]:
# from uuid import uuid4

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = f"LangGraph Demo - {uuid4().hex[0:8]}"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Tool Selection

### Creating the Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Duck Duck Go Web Search](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/ddg_search)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)

In [3]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tool_belt = [DuckDuckGoSearchRun(), ArxivQueryRun()]

### Actioning with Tools

Now that we've created our tool belt - we need to create a process that will let us leverage them when we need them.

We'll use the built-in [`ToolExecutor`](https://github.com/langchain-ai/langgraph/blob/main/langgraph/prebuilt/tool_executor.py) to do so.

In [4]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tool_belt)

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, streaming=True) # # "gpt-4o"   ou    "gpt-3.5-turbo-0125"

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [6]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

In [7]:
functions

[{'name': 'duckduckgo_search',
  'description': 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.',
  'parameters': {'type': 'object',
   'properties': {'query': {'description': 'search query to look up',
     'type': 'string'}},
   'required': ['query']}},
 {'name': 'arxiv',
  'description': 'A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.',
  'parameters': {'type': 'object',
   'properties': {'query': {'description': 'search query to look up',
     'type': 'string'}},
   'required': ['query']}}]

In [8]:
model

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7d646c2e1780>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7d646c2e2e90>, model_name='gpt-3.5-turbo-0125', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', streaming=True), kwargs={'functions': [{'name': 'duckduckgo_search', 'description': 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type': 'object', 'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query']}}, {'name': 'arxiv', 'description': 'A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.', '

## Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [9]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]

## It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [10]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

async def call_model(state):
  messages = state["messages"]
  response = await model.ainvoke(messages)
  return {"messages" : [response]}

async def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = await tool_executor.ainvoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `call_tool` is a node which will call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [11]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [12]:
workflow.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [13]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges("agent",
                               should_continue,
                               {
                                  "continue" : "action",
                                  "end" : END
                               }
                              )

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [14]:
workflow.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [15]:
app = workflow.compile()

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [16]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="O que são mutantes de nível ômega? Forneça fontes em sua resposta.")]} # "What are Omega Level Mutants? Provide sources in your response."

await app.ainvoke(inputs)

{'messages': [HumanMessage(content='O que são mutantes de nível ômega? Forneça fontes em sua resposta.'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"O que são mutantes de nível ômega"}', 'name': 'duckduckgo_search'}}, response_metadata={'finish_reason': 'function_call'}, id='run-17a446bb-3e80-4915-a404-34b94ab69f03-0'),
  FunctionMessage(content='Dentro das HQs da Marvel, os mutantes são colocados numa classificação de acordo com seu nível de poder e o tipo de suas habilidades. Esse sistema possui seis classes distintas: Ômega, Alfa, Beta, Gama, Delta e Épsilon, todos nomeados de acordo com letras do alfabeto grego. A ideia por trás disso é mostrar como as mutações podem ser ... Caso você esteja acompanhando os eventos de X-Men \'97, já se deparou com alguns momentos da animação em que mutantes como Tempestade são classificados como pertencentes ao "nível Ômega ... Tempo estimado de leitura: 11 minutos O Universo Marvel possui mutantes extremam

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "function_call" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "function_call" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [18]:
# "What is QLoRA in Machine Learning? Are their any papers that could help me understand? Once you have that information, can you look up the bio of the first author on the QLoRA paper?"
inputs = {"messages" : [HumanMessage(content="O que é QLoRA em aprendizado de máquina? Há algum documento que possa me ajudar a entender? Depois de ter essas informações, você pode consultar a biografia do primeiro autor do artigo QLoRA?")]}

await app.ainvoke(inputs)

{'messages': [HumanMessage(content='O que é QLoRA em aprendizado de máquina? Há algum documento que possa me ajudar a entender? Depois de ter essas informações, você pode consultar a biografia do primeiro autor do artigo QLoRA?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"QLoRA machine learning"}', 'name': 'arxiv'}}, response_metadata={'finish_reason': 'function_call'}, id='run-27cfdf99-957d-44d1-a90d-2e518c6e3da0-0'),
  FunctionMessage(content='Published: 2023-05-23\nTitle: QLoRA: Efficient Finetuning of Quantized LLMs\nAuthors: Tim Dettmers, Artidoro Pagnoni, Ari Holtzman, Luke Zettlemoyer\nSummary: We present QLoRA, an efficient finetuning approach that reduces memory usage\nenough to finetune a 65B parameter model on a single 48GB GPU while preserving\nfull 16-bit finetuning task performance. QLoRA backpropagates gradients through\na frozen, 4-bit quantized pretrained language model into Low Rank\nAdapters~(LoRA). Our best model family, wh